In [ ]:
# 🚗 Car Parts Image Generator (Stable Diffusion + CSV)
# Google Colab notebook

# --- 1. Setup ---
!pip install diffusers transformers accelerate safetensors pandas

import os
import pandas as pd
from diffusers import StableDiffusionPipeline
import torch
from google.colab import drive

# --- 2. Mount Google Drive ---
# This step requires authentication. Follow the prompts when you run the cell.
drive.mount('/content/drive')

# Change this path to where your CSV is stored on Drive
csv_path = "/content/drive/MyDrive/car_parts/05_01_car_parts_game_style_100.csv"
output_dir = "/content/drive/MyDrive/car_parts/generated_images"
os.makedirs(output_dir, exist_ok=True)

# --- 3. Load CSV ---
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} car parts.")

# --- 4. Load Stable Diffusion model ---
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# --- 5. Generate images ---
for idx, row in df.iterrows():
    part_name = row.get("Part") or row.get("Part Name") or f"part_{idx}"
    part_description = row.get("Description")
    prompt = f"A futuristic, highly detailed, game-style car part: {part_name}: {part_description}"

    try:
        image = pipe(prompt, guidance_scale=7.5).images[0]
        filename = f"{part_name.replace(' ', '_')}.png"
        filepath = os.path.join(output_dir, filename)
        image.save(filepath)
        print(f"✅ Generated {part_name} → {filepath}")
    except Exception as e:
        print(f"❌ Failed for {part_name}: {e}")

print("🎉 Done! All images saved to Google Drive.")